In [1]:
from pathlib import Path
import pandas as pd
import xlwings as xw


In [2]:
def set_file_root(data_column: pd.Series, base_path: Path) -> pd.DataFrame:
    cleaned_data = data_column.str.replace('W:', 'w:')
    cleaned_data = cleaned_data.str.replace(r'\\dkphysicspv1\e$\Gregs_Work', 
                                            'w:', regex=False)
    cleaned_data = cleaned_data.str.replace(str(base_path), '.', regex=False)
    cleaned_data = cleaned_data.str.strip('\\')
    cleaned_data.name = 'Path'
    project_folder = cleaned_data.str.split('\\', 2, expand=True).iloc[:,1]
    project_folder.fillna(base_path.name, inplace=True)
    project_folder[project_folder.str.len() == 0] = base_path.name
    project_folder.name = 'ProjectFolder'
    path_data = pd.concat([cleaned_data, project_folder], axis='columns')
    return path_data

In [3]:
base_path = Path.cwd().parent

In [4]:
data_file = base_path / 'Environment' / 'plan_check_env_analysis.xlsx'
workbook = xw.Book(data_file)

In [5]:
files_sheet = workbook.sheets['WorkspaceContents'].activate()
file_data = xw.load(index=False)
path_data = set_file_root(file_data.Location, base_path)
file_data = pd.concat([file_data, path_data], axis='columns')

file_parts = file_data.Name.str.rsplit('.', 1, expand=True)
file_data['ext'] = file_parts.iloc[:,1]
file_data['Module'] = file_parts.iloc[:,0]
python_files = file_data[file_data.ext == 'py']


python_files = python_files.rename(columns={'Name': 'FileName'})
selected_columns = ['Module', 'FileName', 'ProjectFolder', 
                    'Modified Date', 'Path']
python_files = python_files[selected_columns]

In [6]:
import_sheet = workbook.sheets['Imports Table'].activate()
import_data = xw.load(index=False)

path_data = set_file_root(import_data.Folder, base_path)
import_data = pd.concat([import_data, path_data], axis='columns')
import_data.rename(inplace=True, columns={
    'File': 'PythonCallingFile',
    'ProjectFolder': 'PythonCallingProject',
    'Import Module': 'ImportedModule',
    'Import Functions': 'ImportedFunctions'
    })

selected_columns = ['PythonCallingFile', 'PythonCallingProject', 'Path', 
                    'ImportedModule', 'ImportedFunctions']
import_data = import_data[selected_columns]


In [7]:
old_files = import_data.Path.str.contains('Old files', regex=False)
import_data = import_data[~old_files]

checkpoint_files = import_data.Path.str.contains('checkpoints', regex=False)
import_data = import_data[~checkpoint_files]

old_files = import_data.Path.str.contains('old code', regex=False)
import_data = import_data[~old_files]

In [8]:
import_ref = import_data.merge(python_files, how='left', 
                               left_on=['PythonCallingFile', 'Path'], 
                               right_on=['FileName', 'Path'], 
                               suffixes=('_import', '_file'))

In [9]:
import_ref.columns

Index(['PythonCallingFile', 'PythonCallingProject', 'Path', 'ImportedModule',
       'ImportedFunctions', 'Module', 'FileName', 'ProjectFolder',
       'Modified Date'],
      dtype='object')

In [10]:
python_files.columns

Index(['Module', 'FileName', 'ProjectFolder', 'Modified Date', 'Path'], dtype='object')

In [11]:
module_ref = import_ref.merge(python_files, how='left', 
                               left_on=['ImportedModule', 'PythonCallingProject'], 
                               right_on=['Module', 'ProjectFolder'], 
                               suffixes=('_calling_file', '_imported_module'))

In [12]:
module_ref.columns

Index(['PythonCallingFile', 'PythonCallingProject', 'Path_calling_file',
       'ImportedModule', 'ImportedFunctions', 'Module_calling_file',
       'FileName_calling_file', 'ProjectFolder_calling_file',
       'Modified Date_calling_file', 'Module_imported_module',
       'FileName_imported_module', 'ProjectFolder_imported_module',
       'Modified Date_imported_module', 'Path_imported_module'],
      dtype='object')

In [13]:
selected_columns = ['PythonCallingFile', 'PythonCallingProject', 
                    'Path_calling_file','Modified Date_calling_file', 
                    'ImportedModule', 'FileName_imported_module', 
                    'ProjectFolder_imported_module', 'Path_imported_module', 
                    'Modified Date_imported_module']

In [14]:
ref_sheet = workbook.sheets.add('Import Reference')
xw.view(import_ref, sheet=ref_sheet)

ref_sheet = workbook.sheets.add('Module Reference')
xw.view(module_ref[selected_columns], sheet=ref_sheet)